In [1]:
import os
import ast
import json
import shutil
import platform
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col

tqdm.pandas()
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'

TABLES = 'Output/Tables'
with open(f'{PROCESSED}/pretty_dict.json', 'r', encoding='utf-8') as f:
    pretty_dict = json.load(f)


In [2]:
# %% Load data
df = pd.read_csv(f'{PROCESSED}/hundred_dataset.csv', encoding='utf-8')
df['lland'] = np.log(df['landOwned'] + 1)
df['monast_land_share'] = (df['landOwned']/240)/(df['hundred_val_1524'] + df['landOwned']/240)

# Set np.nan if ind_1831 == agr_1831 == 0
df.loc[(df['ind_1831'] == 0) & (df['agr_1831'] == 0), 'ind_1831'] = np.nan
df.loc[(df['ind_1831'] == 0) & (df['agr_1831'] == 0), 'agr_1831'] = np.nan
df['other_1831'] = 1 - (df['ind_1831'] + df['agr_1831'])



In [21]:
df['pop_growth_1524_1801'] = df['pop01']/df['hundred_count_1524']
df['density_1524'] = df['hundred_count_1524']/df['area']  # people per km^2
a = list(df.columns)
a = [x for x in a if 'pop' in x]
print(a)

['pop01', 'pop_fam', 'pop11', 'pop21', 'pop31', 'pop11_01', 'pop21_11', 'pop31_21', 'pop31_01', 'pop15', 'pop_growth_1524_1801']


In [31]:
# %% Regression time!
for year in [1524, 1543, 1581, 1674, 1840]:
    
    #%% Regression on economic indicators
    result_list = []
    for depvar in ['ind_1831', 'agr_1831', 'other_1831']:
        xvars = ['monast_land_share', f'hundred_master_treatment_{year}', f'hundred_master_control_{year}', 'mean_slope', 'wheatsuit', 'area', 'lspc1525',
                    'distmkt']
        reg_df = df.copy()
        reg_df = reg_df.dropna(subset=['ind_1831', 'agr_1831', 'other_1831'] + xvars)

        y = reg_df[depvar]
        y.rename(pretty_dict, inplace=True)
        x = reg_df[xvars]
        x = sm.add_constant(x)
        x.rename(columns=pretty_dict, inplace=True)
        # Fit the model with heteroskedasticity robust standard errors
        model = sm.OLS(y, x).fit(cov_type='HC3')
        result_list.append(model)
    economic_reg = summary_col(result_list, stars=True,
                                float_format='%0.2f',
                                model_names=['Industry Share', 'Agriculture Share', 'Other Share'],
                                info_dict={'N': lambda x: "{0:d}".format(int(x.nobs))},
                                regressor_order=[pretty_dict[x] for x in xvars]
                                ).as_latex()
    economic_reg = economic_reg.replace(
        '\\end{center}\n\\end{table}\n\\bigskip\nStandard errors in parentheses. \\newline \n* p<.1, ** p<.05, ***p<.01',
        '\\end{center}\n* p<.1, ** p<.05, ***p<.01\n\\end{table}')
    economic_reg = economic_reg.replace('<', '$<$').replace('>', '$>$').replace('\\label{}', '\\label{hundred_' + measure + '_' + str(year) + '}')
    economic_reg = economic_reg.replace('\\begin{center}', '\\begin{center}\n\\resizebox{\\textwidth}{!}{')
    economic_reg = economic_reg.replace('\\end{tabular}', '\\end{tabular}\n}')
    economic_reg = economic_reg.replace('\\begin{table}', '\\begin{table}[H]')
    for table, filename in zip([economic_reg], ['economic_reg']):
        with open(f'{TABLES}/{filename}_hundred_{year}.tex', 'w', encoding='utf-8') as f:
            f.write(table)
        print(table)

\begin{table}[H]
\caption{}
\label{hundred_tot_val_1524}
\begin{center}
\resizebox{\textwidth}{!}{
\begin{tabular}{llll}
\hline
                                               & Industry Share & Agriculture Share & Other Share  \\
\hline
Monastic Land Share                            & -0.29          & 0.17              & 0.12         \\
                                               & (0.20)         & (0.75)            & (0.83)       \\
Treatment Group Share of Hundred Surnames 1524 & -0.14          & 2.41              & -2.28        \\
                                               & (0.92)         & (1.64)            & (1.68)       \\
Control Group Share of Hundred Surnames 1524   & -0.35          & 0.31              & 0.04         \\
                                               & (0.99)         & (2.92)            & (2.57)       \\
Mean Slope                                     & -0.01          & -0.02             & 0.03         \\
                                               & 

In [24]:
# %% Regression time!
for year in [1524, 1543, 1581, 1674, 1840]:
    
    #%% Regression on economic indicators
    result_list = []
    depvar = 'pop_growth_1524_1801'
    xvars = ['monast_land_share', f'hundred_master_treatment_{year}', f'hundred_master_control_{year}', 
             'mean_slope', 'wheatsuit', 'area', 'lspc1525', 'distmkt']
    reg_df = df.copy()
    reg_df = reg_df.dropna(subset=[depvar] + xvars)

    y = reg_df[depvar]
    y.rename(pretty_dict, inplace=True)
    x = reg_df[xvars]
    x = sm.add_constant(x)
    x.rename(columns=pretty_dict, inplace=True)
    # Fit the model with heteroskedasticity robust standard errors
    model = sm.OLS(y, x).fit(cov_type='HC3')
    print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     pop_growth_1524_1801   R-squared:                       0.723
Model:                              OLS   Adj. R-squared:                  0.627
Method:                   Least Squares   F-statistic:                    0.7114
Date:                  Sun, 28 Sep 2025   Prob (F-statistic):              0.679
Time:                          16:04:13   Log-Likelihood:                -88.839
No. Observations:                    32   AIC:                             195.7
Df Residuals:                        23   BIC:                             208.9
Df Model:                             8                                         
Covariance Type:                    HC3                                         
                                                     coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------